In [17]:
!pip install pathway bokeh --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
# upload  dataset

from google.colab import files

uploaded = files.upload()


Saving dataset (1).csv to dataset (1) (1).csv


In [55]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
import pathway as pw

# read dataset
df = pd.read_csv('dataset (1) (1).csv')

# combine date and time into a single datetime column for time-based plotting
df['Time'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')

df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Time
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00,2016-10-04 08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00,2016-10-04 08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00,2016-10-04 09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00,2016-10-04 09:59:00


# Model 1: Baseline Linear Pricing

This is a simple linear pricing formula where the price increases as the occupancy of the parking lot increases.


* Formula :

    Price = Base + α × (Occupancy / Capacity)

* Logic:
        (a) Base price: $10
        (b) Price bounds: $5 to $20
        (c) Scaling factor [α]: 10

  

In [56]:
# set parameters

base_price = 10

min_price = 5

max_price = 20

alpha = 10  # this controls how fast price increases



# calculating price for each row using min and max to bound the result

def bounded_linear_price(occupancy, capacity):
    price = base_price + alpha * (occupancy / capacity)

    #  ensure price is not below min_price
    price = max(price, min_price)

    #  ensure price is not above max_price
    price = min(price, max_price)
    return price

df['Price_Model1'] = df.apply(lambda row: bounded_linear_price(row['Occupancy'], row['Capacity']), axis=1)


In [57]:
# first few results

print(df[['Occupancy', 'Capacity', 'Price_Model1']].head())

   Occupancy  Capacity  Price_Model1
0         61       577     11.057192
1         64       577     11.109185
2         80       577     11.386482
3        107       577     11.854419
4        150       577     12.599653


# Real-Time Visualization for Model 1 (Bokeh)

Plot:

Line plot of price over time for a selected parking lot.

In [59]:
# select a parking lot to visualize (i use systemcodenumber)
lot_id = df['SystemCodeNumber'].iloc[0]
lot_df = df[df['SystemCodeNumber'] == lot_id].copy().sort_values('Time')

# Prepare the data source for Bokeh
source = ColumnDataSource(lot_df)

# Create the figure
p = figure(
    title=f"Real-Time Price (Model 1) for Lot {lot_id}",
    x_axis_label='Time',
    y_axis_label='Price ($)',
    x_axis_type='datetime',
    width=800,
    height=350,
    tools="pan,wheel_zoom,box_zoom,reset,save"
)

# Add the line plot
p.line('Time', 'Price_Model1', source=source, line_width=2, legend_label="Price (Model 1)", color="navy")
p.scatter('Time', 'Price_Model1', source=source, size=5, color="orange", legend_label="Data Points")

p.legend.location = "top_left"
p.legend.click_policy = "hide"

show(p)

# Model 2: Demand-Based Pricing

This model uses a demand function that combines multiple features: occupancy rate, queue length, traffic congestion, special day indicator, and vehicle type. The demand value is normalized and used to adjust the price smoothly.

* Demand Function:
                  Demand= α⋅(Occupancy/Capacity) + β⋅(QueueLength) − γ⋅(TrafficLevel) + δ⋅(IsSpecialDay) + ϵ⋅(VehicleTypeWeight)

* Price Formula:
                Price= BasePrice⋅( 1 + λ⋅{NormalizedDemand} )

* Logic:
        
      (a) Base price: $10
      (b) Price bounds: $5 to $20

In [60]:
# map categorical features to numeric
traffic_map = {'low': 0, 'medium': 1, 'high': 2}
vehicle_map = {'car': 1.0, 'bike': 0.5, 'truck': 2}

df['TrafficNum'] = df['TrafficConditionNearby'].map(traffic_map)
df['VehicleWeight'] = df['VehicleType'].map(vehicle_map)

# weights for each feature (the values taken are made up)
weights = {
    'occupancy': 0.5,   # α
    'queue': 0.3,       # β
    'traffic': 0.2,     # γ
    'special_day': 0.2, # δ
    'vehicle': 0.1      # ϵ
}

# calculataion
occupancy_ratio = df['Occupancy'] / df['Capacity']
raw_demand = (
    weights['occupancy'] * occupancy_ratio +
    weights['queue'] * df['QueueLength'] +
    weights['traffic'] * df['TrafficNum'] +
    weights['special_day'] * df['IsSpecialDay'] +
    weights['vehicle'] * df['VehicleWeight']
)

# normalize demand between 0 and 1
normalized_demand = (raw_demand - raw_demand.min()) / (raw_demand.max() - raw_demand.min())

# computing price
base_price = 10
lamb = 0.8  # λ
price = base_price * (1 + lamb * normalized_demand)

# bound the price between 0.5x and 2x base price
price = price.apply(lambda x: max(min(x, base_price * 2), base_price * 0.5))

df['Price_Model2'] = price



In [61]:
# first few results

print(df[['Occupancy', 'QueueLength', 'TrafficConditionNearby', 'IsSpecialDay', 'VehicleType', 'Price_Model2']].head())

   Occupancy  QueueLength TrafficConditionNearby  IsSpecialDay VehicleType  \
0         61            1                    low             0         car   
1         64            1                    low             0         car   
2         80            2                    low             0         car   
3        107            2                    low             0         car   
4        150            2                    low             0        bike   

   Price_Model2  
0     10.563409  
1     10.567132  
2     11.016717  
3     11.050232  
4     11.031985  


# Real-Time Visualization for Model 2 (Bokeh)

In [62]:
# select the same parking lot (as model 1) for comparison
lot_df2 = df[df['SystemCodeNumber'] == lot_id].copy().sort_values('Time')
source2 = ColumnDataSource(lot_df2)

p2 = figure(
    title=f"Real-Time Price (Model 2) for Lot {lot_id}",
    x_axis_label='Time',
    y_axis_label='Price ($)',
    x_axis_type='datetime',
    width=900,
    height=400,
    tools="pan,wheel_zoom,box_zoom,reset,save"
)

p2.line('Time', 'Price_Model2', source=source2, line_width=2, legend_label="Price (Model 2)", color="green")
p2.scatter('Time', 'Price_Model2', source=source2, size=5, color="orange", legend_label="Data Points")

p2.legend.location = "top_left"
p2.legend.click_policy = "hide"

show(p2)